<a href="https://colab.research.google.com/github/MustiCankan/SpiriteAI/blob/main/SameNew2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OMNIGEN UI Gradio Same v1.2.1

In [ ]:
!pip install gradio
!pip install git+https://github.com/huggingface/diffusers
!pip install -U diffusers transformers accelerate torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 119.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-ql5o3sq5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-ql5o3sq5
  Resolved https://github.com/huggingface/diffusers to commit 680a8ed855fa0bc3191d4f55e20af23d24338244
  Installing build dependencies ... done
  Getti

In [ ]:
import gradio as gr
from diffusers import DiffusionPipeline, FluxPipeline
import torch
from diffusers import OmniGenPipeline
from diffusers.utils import load_image

def load_model(input_selected_model, lora, prompt, step_size, guidance_scale, input_image1, input_image2,image_guidance_scale,seed):

    if input_selected_model == "OmniGen":

        print("OmniGen is loading.")

        pipe = OmniGenPipeline.from_pretrained(
            "Shitao/OmniGen-v1-diffusers",
            torch_dtype=torch.bfloat16
        )
        pipe.enable_model_cpu_offload()
        pipe.to("cuda")
        print("OmniGen is loaded.")

        image = generate_image_with_progress(pipe,prompt, step_size, guidance_scale, input_image1, input_image2,image_guidance_scale,seed)
        print("Image is created ")

    elif input_selected_model == "Flux":
        print("Flux is loading.")
        # Load the model
        pipe2 = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16)
        if lora == "Realism":
            pipe2.load_lora_weights("strangerzonehf/Realism-v3-Flux", weight_name="Realism-v3.safetensors")
        elif lora == "RealBeauty":
            pipe2.load_lora_weights("strangerzonehf/RealBeauty-v3-Flux", weight_name="RealBeauty-v3.safetensors")
        else:
            pass


        # Enable memory optimizations.
        pipe2.enable_model_cpu_offload()
        step_size = int(step_size)
        guidance_scale = float(guidance_scale)
        print("Generating Flux Image...")
        image = pipe2(
        prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=step_size,
        max_sequence_length=256,
        generator=torch.Generator("cpu").manual_seed(seed)
        ).images[0]

        image.save("output.png")
        print("Image is created ")
    return image

def show_image(img):
    print(img)
    return img


def generate_image_with_progress(pipe,prompt, step_size, guidance_scale, input_image1, input_image2,image_guidance_scale,seed):
    step_size = int(step_size)
    guidance_scale = float(guidance_scale)

    input_images = []

    if input_image1 != None:
        input_images.append(load_image(input_image1))
        print("Image 1 is loaded.")
    if input_image2 != None:
        input_images.append(load_image(input_image2))
        print("Image 2 is loaded.")

    #  Check condition if input_images are avaliable
    if len(input_images) > 0:
      print("There are images,so pipeline modified.")
      image = pipe(
          prompt=prompt,
          input_images=input_images,
          height=1024,
          width=1024,
          num_inference_steps=step_size,
          guidance_scale=guidance_scale,
          img_guidance_scale=image_guidance_scale,
          generator=torch.Generator(device="cpu").manual_seed(seed)
          ).images[0]
    else:
    # Running the pipeline
        image = pipe(
          prompt=prompt,
          height=1024,
          width=1024,
          num_inference_steps=step_size,
          guidance_scale=guidance_scale,
          img_guidance_scale=image_guidance_scale,
          generator=torch.Generator(device="cpu").manual_seed(seed)
          ).images[0]

    image.save("output.png")

    return image

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Group():
            gr.Markdown("### Spirit AI Model Extractor (SAME) v1.3.5", elem_id="name_l")

            logo = gr.Image(value="/content/logo2.png", show_label=False, elem_id="logo-image")

            with gr.Accordion("Upload Images", open=False):
              with gr.Column():
                input_image1 =gr.Image(label="Input Image Component 1", type="filepath")
                input_image2 =gr.Image(label="Input Image Component 2", type="filepath")
        with gr.Column():

            prompt = gr.Textbox(label="Prompt")
            pretrain_dropdown = gr.Dropdown(
                ["Flux", "OmniGen"],
                label="Pretrain Models",
                info="Select the pre-trained model"
            )

            lora_dropdown = gr.Dropdown(
                ["Realism", "RealBeauty"],
                label="Loras",
                info="Select the lora Model"
            )

            with gr.Accordion("Advanced options", open=False):
                step_size = gr.Number(label="Step Size", value=50)
                guidance_scale = gr.Number(label="Guidance Scale", value=3)
                image_guidance_scale = gr.Number(label="Image Guidance Scale", value=1.6)
                seed = gr.Number(label="Seed", value=42)
            generate_image_btn = gr.Button(value="Generate Image")
            show_image_btn = gr.Button(value="Show Image")
            examples = gr.Examples(
                examples=["Blonde hair girl", "Bald hair girl",
                          "A woman is holding and prominently showcasing a canned cat food. The woman has [describe appearance if needed]. The reference for the canned cat food is '[input]'. Ensure the can is clearly visible and detailed, with branding and design intact.",
                          "Realistic photo. A young woman sits on a sofa, holding a book and facing the camera. She wears delicate silver hoop earrings adorned with tiny, sparkling diamonds that catch the light, with her long chestnut hair cascading over her shoulders. Her eyes are focused and gentle, framed by long, dark lashes. She is dressed in a cozy cream sweater, which complements her warm, inviting smile. Behind her, there is a table with a cup of water in a sleek, minimalist blue mug. The background is a serene indoor setting with soft natural light filtering through a window, adorned with tasteful art and flowers, creating a cozy and peaceful ambiance. 4K, HD."
                          ],
                inputs=[prompt]
            )
        with gr.Column():
            output_image = gr.Image(label="Generated Image")


        # Custom CSS for resizing the logo
        demo.css = """
    #logo-image img {
      width: 500px;
      height: auto;
      display: block;
      margin: 5px auto 20px auto; /* bottom margin of 20px */
      padding: 8px;
      }

  #name_l  {
    max-width: 100%;
    height: auto;
    display: block;
    margin: 5 auto;
    padding: 8;
    }
    """


    generate_image_btn.click(
        fn=load_model,
        inputs=[pretrain_dropdown, lora_dropdown, prompt, step_size, guidance_scale, input_image1, input_image2,image_guidance_scale,seed],
        outputs=output_image
    )
    show_image_btn.click(
        fn=show_image,
        inputs=input_image1,
        outputs=output_image
    )


demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6f74250ffa0fecebe0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Flux is loading.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

...

(…)pytorch_model-00001-of-00003.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

(…)pytorch_model-00002-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

(…)ion_pytorch_model.safetensors.index.json:   0%|          | 0.00/121k [00:00<?, ?B/s]

vae%2Fconfig.json:   0%|          | 0.00/774 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [ ]:
import torch
from diffusers import FluxPipeline

pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16)
pipe.load_lora_weights("strangerzonehf/Realism-v3-Flux", weight_name="Realism-v3.safetensors")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe.to(device)


# Enable memory optimizations.
pipe.enable_model_cpu_offload()

prompt = "Blonde hair lady holding a coffee in the beach,realistic,8k"
image = pipe(
    prompt,
    guidance_scale=3.5,
    num_inference_steps=28,
    max_sequence_length=256,
    generator=torch.Generator("cpu").manual_seed(0)
).images[0]
image.save("flux-schnell.png")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


[]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[]


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]